In [11]:
import pandas as pd
import os
import numpy as np
import ast

In [12]:
df = pd.read_csv('NLP_input.csv')
df.head()

,Name,Text Data,depression,flattened text,tokenized_text,expanded_text
0,369,"[['okay', 'awesome', 'thank', 'you'], ['are', ...",0,"['okay', 'awesome', 'thank', 'you', 'are', 'yo...","['okay', 'awesome', 'thank', 'you', 'are', 'yo...","['okay', 'awesome', 'thank', 'you', 'are', 'yo..."
1,423,"[['okay'], ['and', 'please'], ['yes'], ['feeli...",0,"['okay', 'and', 'please', 'yes', 'feeling', 'w...","['okay', 'and', 'please', 'yes', 'feeling', 'w...","['okay', 'and', 'please', 'yes', 'feeling', 'w..."
2,436,"[['that', 'was', 'big'], ['yes'], [""I'm"", 'doi...",0,"['that', 'was', 'big', 'yes', ""I'm"", 'doing', ...","['that', 'was', 'big', 'yes', 'I', ""'m"", 'doin...","['that', 'was', 'big', 'yes', 'I', 'am', 'doin..."
3,318,"[['all', 'right'], ['okay'], ['oh', 'actually'...",0,"['all', 'right', 'okay', 'oh', 'actually', 'be...","['all', 'right', 'okay', 'oh', 'actually', 'be...","['all', 'right', 'okay', 'oh', 'actually', 'be..."
4,447,"[['yeah', ""that's"", 'perfectly', 'fine'], [""I'...",0,"['yeah', ""that's"", 'perfectly', 'fine', ""I'm"",...","['yeah', 'that', ""'s"", 'perfectly', 'fine', 'I...","['yeah', 'that', 'is', 'perfectly', 'fine', 'I..."


In [13]:
df=df.drop(columns = ['tokenized_text','expanded_text','flattened text'])

In [14]:
import ast
columns = ['Text Data']

for item in columns:
    df[item] = df[item].apply(lambda x: ast.literal_eval(x))

In [15]:
df.head()

,Name,Text Data,depression
0,369,"[[okay, awesome, thank, you], [are, you, okay,...",0
1,423,"[[okay], [and, please], [yes], [feeling, well]...",0
2,436,"[[that, was, big], [yes], [I'm, doing, fine], ...",0
3,318,"[[all, right], [okay], [oh, actually, before, ...",0
4,447,"[[yeah, that's, perfectly, fine], [I'm, feelin...",0


In [16]:
def process_text_data(text_data):
    # For each sublist, join the words in lowercase with spaces and add a period
    return [' '.join(word.lower() for word in sublist) + '.' for sublist in text_data]

# Apply the function to each entry in the 'Text Data' column
df['Processed Text Data'] = df['Text Data'].apply(process_text_data)
df['Processed Text Data'] = df['Processed Text Data'].apply(lambda x: ' '.join(x))

In [17]:
df.head()

,Name,Text Data,depression,Processed Text Data
0,369,"[[okay, awesome, thank, you], [are, you, okay,...",0,okay awesome thank you. are you okay with this...
1,423,"[[okay], [and, please], [yes], [feeling, well]...",0,okay. and please. yes. feeling well. where are...
2,436,"[[that, was, big], [yes], [I'm, doing, fine], ...",0,that was big. yes. i'm doing fine. mexico. whe...
3,318,"[[all, right], [okay], [oh, actually, before, ...",0,all right. okay. oh actually before that one t...
4,447,"[[yeah, that's, perfectly, fine], [I'm, feelin...",0,yeah that's perfectly fine. i'm feeling great ...


In [18]:
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from time import sleep

In [19]:
  opt = tf.keras.optimizers.Adam(1e-5)
  print(f"this is opt: {opt}")

this is opt: <tf_keras.src.optimizers.adam.Adam object at 0x7f522b6a5e70>


In [20]:
k = 5
seed_values = list(range(6,7))

for seed in seed_values:
    print(f"Running with seed: {seed}")
    data = df
    # Set the seed for Python random module
    random.seed(seed)

    # Set the seed for NumPy
    np.random.seed(seed)
    # Set the seed for TensorFlow
    tf.random.set_seed(seed)
    # Initialize the stratified k-fold cross-validator
    kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True) # ensure the shuffling is the reproducible
    # Load the pre-trained BERT model and tokenizer
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    # Define a loss function and metrics
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]
    # Define an optimizer
    opt = tf.keras.optimizers.Adam(1e-5)
    print(f"this is opt: {opt}")
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    # tf.keras.optimizers.Optimizer(
    # name='adam',
    # learning_rate=0.001,
    # weight_decay=0,
    # clipnorm=None,
    # clipvalue=None,
    # global_clipnorm=None,
    # use_ema=False,
    # ema_momentum=0.99,
    # ema_overwrite_frequency=None,
    # jit_compile=True,
    # )

    # Split the dataset into features and target arrays
    texts = df['Processed Text Data'].values
    labels = data['depression'].values
    fold_results = []
    # i=1
    # Iterate through the k folds

    for fold, (train_val_index, test_index) in enumerate(kf.split(texts, labels)):
      # Set the seed for each fold (to ensure the reproducibility of each fold, these are fold-specific seeds)
      tf.random.set_seed(seed)
      np.random.seed(seed)
      random.seed(seed)
      # Split train_val set to train and validation sets in 8:2 ratio
      train_index, valid_index = train_test_split(train_val_index, test_size=0.2, stratify=labels[train_val_index], random_state=seed) #ensure the splitting is reproducible
      print(f"Fold {fold + 1}/{k}")
      # Create train, validation, and test datasets for this fold
      train_texts, train_labels = texts[train_index], labels[train_index]
      valid_texts, valid_labels = texts[valid_index], labels[valid_index]
      test_texts, test_labels = texts[test_index], labels[test_index]
      # Tokenize the input texts for the fold
      train_inputs = tokenizer(list(train_texts), padding=True, truncation=True, return_tensors="tf")
      valid_inputs = tokenizer(list(valid_texts), padding=True, truncation=True, return_tensors="tf")
      test_inputs = tokenizer(list(test_texts), padding=True, truncation=True, return_tensors="tf")
      # Define TensorFlow Datasets for training and validation
      train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_inputs), train_labels))
      valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_inputs), valid_labels))
      test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_inputs), test_labels))
      # Compile the model for this fold
      model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)
      # Train the model for this fold
      batch_size = 8
      num_epochs = 50
      model.fit(train_dataset.shuffle(100).batch(batch_size), epochs=num_epochs, validation_data=valid_dataset.batch(batch_size))
      # Evaluate the model on the validation data for this fold
      loss, accuracy = model.evaluate(valid_dataset.batch(batch_size))
      print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")
      model_save_path = f"../sample_data/model/model{fold + 1}"
      model.save_pretrained(model_save_path)
      # Create a DataFrame for test data
      test_data_df = pd.DataFrame({'Case':test_index,'text': test_texts, 'label': test_labels})
      # Save the test data to a CSV file
      test_data_csv_path = f"../sample_data/dataset/test{fold + 1}.csv"
      test_data_df.to_csv(test_data_csv_path, index=False)
    sleep(2)

    final_results = []
    for fold in range(k):
        # Load the model for the current fold
        model_path = f"../sample_data/model/model{fold + 1}"
        loaded_model = TFBertForSequenceClassification.from_pretrained(model_path)
        # Load the test data for the current fold
        data_path = f"../sample_data/dataset/test{fold + 1}.csv"
        data = pd.read_csv(data_path)

        # Prepare to store results
        predictions = []
        predicted_probabilities_list0 = []
        predicted_probabilities_list1 = []
        # predicted_probabilities_list2 = []
        # predicted_probabilities_list3 = []
        # Iterate over each row in the data
        for text in data['text']:
            test_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
            prediction = loaded_model(test_inputs)
            predicted_probabilities = tf.nn.softmax(prediction.logits, axis=1).numpy()
            predicted_class = np.argmax(predicted_probabilities, axis=1)

            predictions.append(predicted_class[0])
            predicted_probabilities_list0.append(predicted_probabilities[0][0])
            predicted_probabilities_list1.append(predicted_probabilities[0][1])
            # predicted_probabilities_list2.append(predicted_probabilities[0][2])
            # predicted_probabilities_list3.append(predicted_probabilities[0][3])
        # Store results in the DataFrame
        data['predicted_class'] = predictions
        data['Fold_Model'] = fold + 1
        data['prob_class_0'] = predicted_probabilities_list0
        data['prob_class_1'] = predicted_probabilities_list1
        # data['prob_class_2'] = predicted_probabilities_list2
        # data['prob_class_3'] = predicted_probabilities_list3

        # Append the results of this fold to the final results
        final_results.append(data)

    # Concatenate all results into a single DataFrame
    final_result_data = pd.concat(final_results)
    sleep(2)
    final_result_data.to_csv(f"../sample_data/dataset/result_depression_epoch50.csv")
    sleep(2)
    # datasets = []
    # for i in range(1, 15):
    file_path = f"../sample_data/dataset/result_depression_epoch50.csv"
    dataset = pd.read_csv(file_path)
    # datasets.append(dataset)
    # merged_df = pd.concat(datasets, ignore_index=True)
    # merged_df['label'].value_counts()
    # final_merged_dfN=merged_df
    label_col = 'label'
    prob_columns = ['prob_class_0', 'prob_class_1']  # Add more if needed
    def calculate_roc_auc(data, label_col, prob_columns, class_label):
        # Drop rows with NaN values in label or probability columns
        data = data.dropna(subset=[label_col] + prob_columns)
        actual = (data[label_col] == class_label).astype(int)
        predicted_prob = data[prob_columns].loc[:, f'prob_class_{class_label}']
        fpr, tpr, _ = roc_curve(actual, predicted_prob)
        auc_score = auc(fpr, tpr)
        return fpr, tpr, auc_score

    # Calculate mean ROC AUC for all classes
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.zeros_like(mean_fpr)
    mean_auc_values = []

    # Iterate over each class
    for class_label in range(len(prob_columns)):
        fpr, tpr, auc_score = calculate_roc_auc(dataset, label_col, prob_columns, class_label)
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
        mean_auc_values.append(auc_score)
        print(f'AUC for Class {class_label}: {auc_score}')

    # Calculate mean TPR and AUC
    mean_tpr /= len(prob_columns)
    mean_auc = np.mean(mean_auc_values)
    print(f'Mean AUC across all classes: {mean_auc}')

Running with seed: 6


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


this is opt: <tf_keras.src.optimizers.adam.Adam object at 0x7f50fd434bb0>
Fold 1/5
Epoch 1/50
 2/18 [==>...........................] - ETA: 2:17 - loss: 0.7149 - accuracy: 0.5625

KeyboardInterrupt: 